Sta 663 - Statistical Computing and Computation - Midterm 2
-----------
Due Wednesday, April 20th by 5:00 pm.



## Setup

In [1]:
# Load necessary packages
import torch
import math


## Task 1 - `newton()` function

For the Newton function, I began by creating a `hess_check()` function that takes the hessian as an input and ensures the hessian is positive definite. It does this by using the `torch.linalg.cholesky()` function to check for positive definiteness, and in the case this does not occur, it adds a small value to the diagonal and tries again. It will perform 10 iterations before throwing an error. Moving to the `newton()` function, it takes a theta initial guess and a function to optimize, while the rest of the inputs have set values. While in the range of maximum iterations the `newton()` evaluates the function at theta, calculates the gradient, and checks `if max(abs(g)) < (abs(f0)+fscale)*tol`. In the event that this occurs the function has been optimized, `newton()` breaks and a dictionary of outputs including function value at minimum, theta values at minimum, number of iterations to reach minimum, and gradient at minimum is returned. If the function has not been optimized the hessian will be calculated, `hess_check()` will be performed, and a step calculated according to Newton's method will be taken. If the step does not provide improvement it will be halved until it does. However an error will be thrown if any of the following occur: maximum iterations are reached without reaching the optimum, the function evaluated at a theta is not finite, or no improvement to the step occurs after the maximum half steps has occured. 

In [2]:
# Check hessian for positive definiteness
def hess_check(hess, max_it = 10):
    # set initial values
    iteration = 0
    positive_def = False
    value = 1e-9

    while not positive_def:          
        try:
            # test if hessian is positive definite
            torch.linalg.cholesky(hess)
            positive_def = True
        except:
            # count iteration
            iteration = iteration + 1
            
            # raise error if iteration > max_it
            if iteration > max_it:
                raise RuntimeError("Max iterations reached without reaching positive definiteness")
            
            # calculate max absolute value of hessian
            hess_max = torch.max(torch.abs(hess))
            # multiply identity matrix by hess_max and some small value, initially 1e-8
            hess_add = torch.eye(hess.shape[0], hess.shape[0]) * value * hess_max * 10
            # calculate new hess by adding hess_add
            hess = hess + hess_add
            
    return(hess)
            

In [3]:
# Optimize function using Newton's Method
def newton(theta, f, tol = 1e-8, fscale = 1.0, maxit = 100, max_half = 20):
    iteration = 0
    
    for i in range(maxit + 1): 
        # calculate initial values
        f0 = f(theta)
        g = torch.autograd.functional.jacobian(f, theta)
        
        # check if optimization has been reached
        if max(abs(g)) < (abs(f0)+fscale)*tol:
            break
        
        # count iteration
        iteration = iteration + 1
        
        # raise error if iteration > maxit
        if iteration > maxit:
            raise RuntimeError("max iterations reached without convergence")
        
        # calculate hessian and ensure it is positive definite
        hess = torch.autograd.functional.hessian(f, theta)
        hess_check(hess)
        
        # calculate step
        step = -torch.matmul(torch.inverse(hess), g)
        
        half = 0
        
        for j in range(max_half + 1):
            # calculate new theta and evaluate function at new theta
            new_theta = theta + step
            new_f = f(new_theta)
            
            # raise error if function is not finite
            if torch.isfinite(new_f) is False:
                raise ValueError("function is not finite")
                
            # break if step made improvement
            if new_f <= f0:
                break
                
            # count half iteration
            half = half + 1
            
            # raise error if half iterations exceeds max_half
            if half > max_half: 
                raise RuntimeError("step fails to reduce the objective after max_half iterations")
            
            # divide step by 2 if no improvement occurred
            step /= 2
        
        # set updated theta values
        theta = new_theta
    
    # define output
    dict_output = {"f" : f(theta), "theta" : theta, "iter" : iteration, "grad" : g}
            
    return(dict_output)

## Task 2 - Minimization examples


### 1. Quadratic function


In [4]:
## Objective function implementation
quad = lambda thetas: thetas[0]**2 - 2*thetas[0] +2*thetas[1]**2 + thetas[1] + 3 

In [5]:
# Theta Evaluations
theta1 = torch.tensor([0,0], dtype = torch.double)
newton(theta = theta1, f = quad)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [6]:
# Theta Evaluation
theta2 = torch.tensor([10,42], dtype = torch.double)
newton(theta = theta2, f = quad)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [7]:
# Theta Evaluation
theta3 = torch.tensor([45, 45], dtype = torch.double)
newton(theta = theta3, f = quad)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

### 2. Rosenbrock's function

In [8]:
## Objective function implementation
rose = lambda thetas: 10*(thetas[1] - thetas[0]**2)**2 + (1 - thetas[0])**2

In [9]:
# Theta Evaluation
theta1 = torch.tensor([0,0], dtype = torch.double)
newton(theta = theta1, f = rose)

{'f': tensor(1.3559e-31, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 9,
 'grad': tensor([-4.6629e-15,  2.2204e-15], dtype=torch.float64)}

In [10]:
# Theta Evaluation
theta2 = torch.tensor([10,42], dtype = torch.double)
newton(theta = theta2, f = rose)

{'f': tensor(2.3562e-28, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 25,
 'grad': tensor([ 1.0791e-13, -3.9968e-14], dtype=torch.float64)}

In [11]:
# Theta Evaluation
theta3 = torch.tensor([45, 45], dtype = torch.double)
newton(theta = theta3, f = rose)

{'f': tensor(6.2925e-23, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 60,
 'grad': tensor([ 1.0157e-10, -4.9396e-11], dtype=torch.float64)}


### 3. Poisson regression likelihood

In [12]:
x = [
   0.11, -0.06, -0.96, -0.48, -0.59, -0.42, -0.15,  1.14, 0.94, 
  -0.86, -0.08,  1.00, -2.01,  2.17, -0.20,  0.82, -0.13, 0.26, 
   0.22,  1.05
]

y = [4, 2, 4, 1, 1, 3, 4, 5, 7, 3, 5, 7, 0, 4, 2, 7, 3, 3, 2, 8]

In [13]:
## Objective function implementation
def pois_func(thetas):
    val = 0
    for i in range(20):
        l_lam = thetas[0] + thetas[1]*x[i]
        l_val = y[i] * l_lam - torch.exp(l_lam) - math.log(math.factorial(y[i]))
        val = val + l_val
        
    return(-val)

In [14]:
# Theta Evaluation
theta1 = torch.tensor([1,1], dtype = torch.double)
newton(theta = theta1, f = pois_func)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 5,
 'grad': tensor([6.2172e-15, 3.8788e-15], dtype=torch.float64)}

In [15]:
# Theta Evaluation
theta2 = torch.tensor([10,4.2], dtype = torch.double)
newton(theta = theta2, f = pois_func)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 20,
 'grad': tensor([9.6183e-08, 1.3595e-07], dtype=torch.float64)}

In [16]:
# Theta Evaluation
theta3 = torch.tensor([4.5, 4.5], dtype = torch.double)
newton(theta = theta3, f = pois_func)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 16,
 'grad': tensor([9.7700e-15, 9.8740e-15], dtype=torch.float64)}